##1.Install and Import Dependencies

In [48]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113


In [49]:
#Let's install other dependencies
!pip install pandas numpy transformers requests beautifulsoup4 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup 
import re

##2.Instatiate Model

In [51]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")   # Loding a tokenizer

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')  # Loading a Model

##3.Encode and Calculate Sentiment

In [52]:
tokens = tokenizer.encode("I hate this food, absolutely the worst", return_tensors='pt')

In [53]:
#tokens

In [54]:
#tokenizer.decode(tokens[0])

In [55]:
result = model(tokens)

In [56]:
result

SequenceClassifierOutput([('logits',
                           tensor([[ 5.1580,  1.5413, -1.0141, -2.9975, -1.8733]],
                                  grad_fn=<AddmmBackward0>))])

In [57]:
result.logits

tensor([[ 5.1580,  1.5413, -1.0141, -2.9975, -1.8733]],
       grad_fn=<AddmmBackward0>)

In [58]:
torch.argmax(result.logits) # it gives the maximum value from a target function.

tensor(0)

In [59]:
int(torch.argmax(result.logits))+1 

1

##4.Collect Reviews

In [60]:
r = requests.get('https://www.yelp.com/biz/izakaya-hon-san-francisco-2')  #request for grabbing a webpage
soup = BeautifulSoup(r.text,'html.parser')  #
regex = re.compile('.*comment*') # extracting a specific component
results = soup.find_all('p', {'class': regex}) 
reviews = [result.text for result in results]

In [61]:
reviews

["Best sushi I've had in forever. Two of us shared sashimi, two rolls, and beef skewers. I couldn't quite decide from menu and ordered two items I saw on Yelp (sunrise and crab & salmon roll), really enjoyed them both! Only remembered to take pic of one.The beef skewer was just okay for me. The beef was more well done than I like and a little pepper heavy. \xa0Not worth filling up on this when you can eat more sushi. Excellent service too. Prices are a little high, but justified by the high quality.",
 "Everything was fantastic. Service, menu, food, drinks. I'd give 5 stars, but my stomach didn't agree and, well, it knows better when it comes to quality. No poisoning, just an overall rejection. Experience was so positive I'm up for rolling the dice again, which will either bring it up to 5 stars or down to 3, depending on outcome.",
 "My experience at Izakaya Hon was with a group of 20 for a work dinner. We made a prior reservation and they gave us the back room which perfectly fit our

##5.Load Reviews into DataFrame and Score

In [62]:
import numpy as np
import pandas as pd

In [63]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [64]:
df['review'].iloc[0]

"Best sushi I've had in forever. Two of us shared sashimi, two rolls, and beef skewers. I couldn't quite decide from menu and ordered two items I saw on Yelp (sunrise and crab & salmon roll), really enjoyed them both! Only remembered to take pic of one.The beef skewer was just okay for me. The beef was more well done than I like and a little pepper heavy. \xa0Not worth filling up on this when you can eat more sushi. Excellent service too. Prices are a little high, but justified by the high quality."

In [65]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [66]:
sentiment_score(df['review'].iloc[1])

3

In [67]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [68]:
df

,review,sentiment
0,Best sushi I've had in forever. Two of us shar...,5
1,"Everything was fantastic. Service, menu, food,...",3
2,My experience at Izakaya Hon was with a group ...,5
3,Food was delectable and delicious. Came for a ...,5
4,When you're craving Japanese but you can't dec...,2
5,Highly recommend it if you are in the Union Sq...,5
6,"The food, service and the ambiance is amazing....",5
7,Great spot! Really fresh fish and high quality...,5
8,"It was late night around the area, all the oth...",2
9,This is my fourth trip to Izakaya Hon and it r...,5


In [69]:
df['review'].iloc[3]

'Food was delectable and delicious. Came for a friends birthday party and this place knows how to host. Ordered every item on the Nama list and each one got better and better. Not only were the vibes immaculate, but the crew was nothing short of spectacular! Please come here. Party of 1 or party of 10, this is the place to be!!'